In [1]:
''' 
for the convinience, changing type of flower to ingteger
setosa      0
versicolor  1 
virginica   2
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
'''

import numpy as np
import pandas as pd


data = pd.read_csv("./iris.data")

data.head(10)

,sepal.lenth,sepal.width,petal.lenth,petal.width,type
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
7,5.0,3.4,1.5,0.2,0
8,4.4,2.9,1.4,0.2,0
9,4.9,3.1,1.5,0.1,0


Build Node class

In [2]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor 
        feature index : sepal and petal
            threshold : best split data of feature
                left  : left node
                right : node
                info_gain = information gain
        '''      
        # for decision node
        self.feature_index = feature_index 
        self.threshold = threshold
        self.left = left 
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

Tree Class

Decision tree is very similar with binary search tree which we have studied at Data structure.
So the concept of algorithm is as follows

1.Recursive Partitioning
Recursive Partitoning is to select a Partitioning point so that the homogeneity of newly created child nodes is maximized by Partitioning.<br>
Homogeneity is maximized, which means that impurities are minimized.<br>
Impurity can be determined by the Gini coefficient and variance. This algorithm will use the Gini coefficient.<br>

2.prunning 
To prevent overfitting due to too many branches, pruning that combines sub-nodes and parent nodes should be carried out.<br>
Overfitting can be prevented by adjusting the maximum depth.<br>

max_depth: maximum depth of the tree. If we set it to None, the tree will grow until all the leaves are pure or the hyperparameter min_samples_split has been reached.<br>

min_samples_split: indicates the minimum number of observations a sheet must have to continue creating new nodes.<br>

min_information_gain: the minimum amount the Information Gain must increase for the tree to continue growing.<br>

In [3]:

class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        # initialize the root of the tree 
        """ 
                root
            nodeL   nodeR    
        """
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        
        ''' 
        recursive function to build the tree
        X : attributes(sepal.lenth to petal.width)  
        Y : type (label)
        num_samples : row of feature
        num_features : column of feature
        ''' 
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X) #row and colum of features 
        #print(' sample:',num_samples, ' feature:',num_features," Class:",Y)
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            #print("best split:",best_split)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split 
                  1. take loop and list every feature of index
                  2. list every value of features
                  3. split dataset according to unique of feature values
                  4. dataset value could be null, so if dataset is not null compute impormation gain
                  5. information gain is as better as small, if current gain > max gain , change index
        '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        #print('num_features:',num_features)
        # loop over all the features
        for feature_index in range(num_features): 
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data 
            if feature index is smaller than threshold(according to maximum gain), set in left array (node)
            if feature index is bigger than threshold, set in right array (node)
        '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            ratio = len(y[y == cls]) / len(y)
            entropy += -ratio * np.log2(ratio)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            ratio = len(y[y == cls]) / len(y)
            gini += ratio**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("Feature"+str(tree.feature_index), "<=", tree.threshold, " Info_Gain:", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [4]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)

from CHAE_ML import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split.split(X, Y, test_size=0.2,datatype='numpy')

x_train: (120, 4) , x_test: (30, 4) ,y_train: (120, 1),y_test: (30, 1) 


In [5]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=2)
classifier.fit(X_train,Y_train)
classifier.print_tree()

Feature2 <= 1.9  Info_Gain: 0.38690476190476186
 left:0.0
 right:Feature2 <= 4.9  Info_Gain: 0.3284742468415938
  left:Feature0 <= 4.9  Info_Gain: 0.01957517700957936
    left:1.0
    right:1.0
  right:Feature3 <= 1.7  Info_Gain: 0.10884353741496589
    left:1.0
    right:2.0


![title](./photo/decision_tree_result.jpg)

In [6]:
Y_pred = classifier.predict(X_test) 
#from sklearn.metrics import accuracy_score
from CHAE_ML import accuracy
accuracy.score(Y_test, Y_pred)

0.7333333333333333

seems accuracy score is pretty low. this is because dataset is not splited<br>
let's try again with proposal data split

In [7]:
#from sklearn.model_selection import train_test_split
print(type(X))
X_train, X_test, Y_train, Y_test = train_test_split.split(X, Y, test_size=0.2,shuffle=True,datatype='ndarray')

<class 'numpy.ndarray'>
data shuffle complete
x_train: (120, 4) , x_test: (30, 4) ,y_train: (120, 1),y_test: (30, 1) 


In [8]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=2)
classifier.fit(X_train,Y_train)
classifier.print_tree()

Y_pred = classifier.predict(X_test) 
accuracy.score(Y_test, Y_pred)

Feature2 <= 1.9  Info_Gain: 0.3413247863247863
 left:0.0
 right:Feature3 <= 1.7  Info_Gain: 0.3617998810306502
  left:Feature2 <= 4.9  Info_Gain: 0.12532328246613972
    left:1.0
    right:2.0
  right:Feature2 <= 4.8  Info_Gain: 0.01697530864197544
    left:2.0
    right:2.0


0.9666666666666667

we can see the accuracy socre is 96.6% data is classified nicely.

In [9]:
#code for train test and accuracy

class train_test_split:
    def __init__(self,data,target,test_size,datatype,shuffle=False):
        self._data = data
        self._target = target
        self._test_size = test_size
        self.shuffle = shuffle
        self.datatype = datatype
        #   self._xtrain = x_train
        #   self._xtest = x_test
        #   self._ytrain = y_train
        #   self._ytest = y_test
    @classmethod
    def split(self, data ,target ,test_size,shuffle=False,datatype='df'):
        #data.reset_index(drop=self.shuffle, inplace=self.shuffle) #데이터를 일단 섞어주기
        if(shuffle==True):
            if type(data) is np.ndarray:
                s = np.arange(data.shape[0])
                np.random.shuffle(s)
                data=data[s]
                target=target[s]
                print('data shuffle complete')       
            else:#(type(data) is DataFrame and shuffle is True):
                s = np.arange(data.value_counts.shape[0])
                np.random.shuffle(s)
                data=data[s]
                target=target[s]
                print('data shuffle complete')       


        if(datatype== 'df'):
            x_train = data.iloc[:round(len(data)*(1-test_size)),:]#0~0.7
            y_train = target.iloc[:round(len(target)*(1-test_size)),]
            x_test = data.iloc[round(len(data)*(1-test_size)):,:]#0.7~1
            y_test = target.iloc[round(len(target)*(1-test_size)):,]#0.7~1
        else:
            x_train = data[:round(len(data)*(1-test_size)),:]#0~0.7
            y_train = target[:round(len(target)*(1-test_size)),]
            x_test = data[round(len(data)*(1-test_size)):,:]#0.7~1
            y_test = target[round(len(target)*(1-test_size)):,]#0.7~1
        # x_train = x_train.to_numpy()
        # y_train = y_train.to_numpy()
        # x_test = x_test.to_numpy()
        # y_test = y_test.to_numpy()
        print("x_train: {} , x_test: {} ,y_train: {},y_test: {} ".format(x_train.shape, x_test.shape, y_train.shape, y_test.shape))

        return x_train , x_test , y_train , y_test 


class accuracy:
    def __init__(self,y_test,y_pred):
        self.y_pred = y_test
        self.y_data = y_pred
    @classmethod
    def score(self,y_test,y_pred):
        acc=np.mean(y_test==y_pred)
        return acc

